In [1]:
### if colab 
# from google.colab import drive
# drive.mount("/content/drive")
# %cd /content/drive/MyDrive/Capstone/Data

#### Dependencies

In [2]:
import pandas as pd
import numpy as np

In [3]:
def add_col(subdata, Manufacturing_X, Manufacturing_y):
      dic_tab1 = {}
      n = subdata.shape[1]
      for i in range(2, n-1):
        dic_tab1[subdata.columns[i]] = Manufacturing_X[i - 2]
      dic_tab1[subdata.columns[n-1]] = Manufacturing_y[0]
      return dic_tab1

#### Data - Manufacturing_value

In [4]:
data = pd.read_excel('raw/Tab1.xlsx')
data = data[data['Country Code'].notna()]
Country_Codes = list(np.unique(data['Country Code']))

In [5]:
data.head()

,Country Name,Country Code,Time,Time Code,"Employment in industry, male (% of male employment) (modeled ILO estimate) [SL.IND.EMPL.MA.ZS]",High-technology exports (% of manufactured exports) [TX.VAL.TECH.MF.ZS],Research and development expenditure (% of GDP) [GB.XPD.RSDV.GD.ZS],Researchers in R&D (per million people) [SP.POP.SCIE.RD.P6],Population ages 15-64 (% of total population) [SP.POP.1564.TO.ZS],Population density (people per sq. km of land area) [EN.POP.DNST],...,"Net trade in goods and services (BoP, current US$) [BN.GSR.GNFS.CD]","Air transport, freight (million ton-km) [IS.AIR.GOOD.MT.K1]",Container port traffic (TEU: 20 foot equivalent units) [IS.SHP.GOOD.TU],"Railways, goods transported (million ton-km) [IS.RRS.GOOD.MT.K6]","Industry (including construction), value added (current US$) [NV.IND.TOTL.CD]",GDP per capita (current US$) [NY.GDP.PCAP.CD],Final consumption expenditure (current US$) [NE.CON.TOTL.CD],Firms using banks to finance investment (% of firms) [IC.FRM.BNKS.ZS],"New business density (new registrations per 1,000 people ages 15-64) [IC.BUS.NDNS.ZS]","Manufacturing, value added (% of GDP) [NV.IND.MANF.ZS]"
0,Canada,CAN,1960.0,YR1960,..,..,..,..,58.574017,..,...,-506965097.119614,..,..,..,..,2259.294285,..,..,..,..
1,Canada,CAN,1961.0,YR1961,..,..,..,..,58.446392,2.037903,...,-212120980.730524,..,..,..,..,2240.433039,32695764216.838642,..,..,..
2,Canada,CAN,1962.0,YR1962,..,..,..,..,58.432887,2.07616,...,-95384130.624447,..,..,..,..,2268.585346,33158772309.478661,..,..,..
3,Canada,CAN,1963.0,YR1963,..,..,..,..,58.522108,2.115198,...,309278759.858606,..,..,..,..,2374.498448,35119813036.783798,..,..,..
4,Canada,CAN,1964.0,YR1964,..,..,..,..,58.697664,2.155463,...,563628154.48225,..,..,..,..,2555.111146,37861761304.363205,..,..,..


In [6]:
### Factors for attractiveness in Manufacturing 
Manufacturing_X = [
  'Employment_in_industry_male',
  'High_technology_exports',
  'Research_development_expenditure',
  'Researchers in R&D',
  'Population_labor_rate',
  'Population_density',
  'Foreign_direct_investment',
  'Net_trade_in_goods_services',
  'Air_freight_million_ton_km',
  'Container_port_traffic_TEU',
  'Railways_goods_trans_million_ton-km',
  'Industry_value_added_current_USD',
  'GDP_per_capita_current_USD',
  'Final_consumption',
  'Access_finance',
  'New_business_density'
]

Manufacturing_y = ['Manufacturing_value added_%_of_GDP']

In [7]:
with pd.ExcelWriter("cleaned/Manufacturing_all.xlsx") as writer:  
  for c in Country_Codes:
    subdata = data[data['Country Code'] == c].reset_index().drop(['index', 'Time'], axis = 1)
    subdata['Year'] = subdata['Time Code'].apply(lambda x: str(x)[2:])
    subdata = subdata.set_index('Year').drop(['Time Code'], axis = 1)
    dic_tab1 = add_col(subdata, Manufacturing_X, Manufacturing_y)
    subdata = subdata.rename(columns = dic_tab1).drop(['Country Name', 'Country Code'], axis = 1) \
            .replace('..', np.nan)
    subdata.to_excel(writer, sheet_name=c)

#### Data - Service_value

In [8]:
data2 = pd.read_excel('raw/Tab2.xlsx')
data2 = data2[data2['Country Code'].notna()]
Country_Codes = list(np.unique(data2['Country Code']))

In [9]:
### Factors for attractiveness in Service 
Service_X = [
 'Employment_in_services_male',
 'Urban_population_growth', 
 'Net_trade_in_goods_services', 
 'Foreign_direct_investment',
 'Mobile_cellular_subscriptions', 
 'Secure_Internet_servers',
 'School_enrollment_tertiary',
 'Population_labor_rate', 
 'Population_density', 
 'Access_electricity',
 'Industry_value_added_current_USD',
 'Employment_in_industry_%_of_total_employment',
 'GDP_per_capita_current_USD',
 'Final_consumption',
 'Primary_completion_rate',
 'Educational_attainment',
 'Individuals_using_the_Internet_%_of_population',
 'Poverty_headcount_ratio',
 'Access_finance',
 'New_business_density'
 ]

Service_y = ['Services_value_added_%_of_GDP']

In [10]:
with pd.ExcelWriter("cleaned/Service_all.xlsx") as writer2:  
  for c in Country_Codes:
    subdata = data2[data2['Country Code'] == c].reset_index().drop(['index', 'Time'], axis = 1)
    subdata['Year'] = subdata['Time Code'].apply(lambda x: str(x)[2:])
    subdata = subdata.set_index('Year').drop(['Time Code'], axis = 1)
    dic_tab2 = add_col(subdata, Service_X, Service_y)
    subdata = subdata.rename(columns = dic_tab2).drop(['Country Name', 'Country Code'], axis = 1) \
            .replace('..', np.nan)
    subdata.to_excel(writer2, sheet_name=c)